In [ ]:
# default_exp dash

# Dash

> Plotly Dash application to visualize and recommendations

https://plot.ly/dash/

In [ ]:
#hide
from nbdev.export import *

In [ ]:
#export
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from functools import lru_cache
from pathlib import Path

import pandas as pd
import numpy as np
import requests

from word2vec_recommender.core import *
from word2vec_recommender.recommender import KnnRecommender
from word2vec_recommender.tmdb import load_api_key, TmdbApi, MovieData

Load Tmdb Api

In [ ]:
#export
def create_dash_app(dataset_path: Path, model_path: Path, dash_params: dict = None):
    # load dataframes
    movies_df = pd.read_csv(dataset_path / "movies.csv")
    links_df = pd.read_csv(dataset_path / "links.csv")

    # create TMDB API
    movielens_to_tmdb_lookup = { 
        int(movie_id):int(tmdb_id)  
        for movie_id, tmdb_id in links_df.set_index("movieId")["tmdbId"].to_dict().items() 
        if movie_id > 0 and tmdb_id > 0 
    }
    api = TmdbApi(api_key=load_api_key(), movielens_to_tmdb_lookup=movielens_to_tmdb_lookup)

    # load model
    with open(model_path / "embeddings.pkl", "rb") as f:
        embeddings = np.load(f)
    with open(model_path / "words_index.pkl", "rb") as f:
        word_indexes = np.load(f)
    movie_id_to_index_lookup = {int(movie_id):idx for idx, movie_id in enumerate(word_indexes)}
    knn_recommender = KnnRecommender(
        word_indexes=word_indexes, 
        embeddings=embeddings)
    knn_recommender.fit()

    if not dash_params:
        dash_params={}
    app = dash.Dash(__name__, **dash_params)

    # base layout
    app.layout = html.Div(children=[
        html.H1(children='MovieLens Recommender based on Word2Vec'),

        html.Div(children='''
            Dash: A web application framework for Python.
        '''),
        dcc.Dropdown(
            id="movie-id",
            options=movies_df[["title", "movieId"]].apply(lambda r:  {"value": r["movieId"], "label": r["title"]} ,axis=1).tolist(),
        ),
        html.Div(id="selected-movie-id", children="")
    ])

    def movie_card(movie: MovieData, seed: bool = False) -> html.Div:
        description = f"{movie.title} (id={movie.movie_id}, tmdb_id={movie.tmdb_id})"
        return html.Div(children=[
            html.H2(description) if seed else html.H3(description),
            html.Img(src=movie.image_url)
        ])

    @app.callback(
        Output(component_id='selected-movie-id', component_property='children'),
        [Input(component_id='movie-id', component_property='value')]
    )
    def update_output_div(movie_id) -> html.Div:
        if not movie_id:
            return html.Div(children="No movie selected")
        print(movie_id)
        movie = api.fetch_movie_data_by_movielens_id(int(movie_id))
        if not movie:
            return html.Div(children="Movie not found on TMDB")
        movie_index = movie_id_to_index_lookup.get(int(movie_id))
        if not movie_index:
            return html.Div(children="No embeddings for movie")
        movie_recs = knn_recommender.recommend_by_index(movie_index) or []
        movies = [ api.fetch_movie_data_by_movielens_id(rec.movie_id) for rec in movie_recs ]
        return html.Div(children=[
            movie_card(movie, seed=True),
            html.Div(children=[
                html.H2("Recommendations"),
                *[movie_card(m) for m in movies]
                ])
            ]) 
    
    return app

In [ ]:
#hide
app = create_dash_app(dataset_path=Path("./data/ml-latest-small/"), model_path=Path("./data/out"))
app.run_server(debug=False, port=9998)

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:9998/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Dec/2019 11:17:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:17:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:17:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:17:51] "GET /_favicon.ico?v=1.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:17:51] "POST /_dash-update-component HTTP/1.1" 200 -
1
>tmdb_movie_id 862
>tmdb_movie_id 10545
>tmdb_movie_id 687
>tmdb_movie_id 13
>tmdb_movie_id 1934
>tmdb_movie_id 177572
>tmdb_movie_id 10451
>tmdb_movie_id 9598
>tmdb_movie_id 863
>tmdb_movie_id 10020
127.0.0.1 - - [20/Dec/2019 11:17:55] "POST /_dash-update-component HTTP/1.1" 200 -
>tmdb_movie_id 812
